In [64]:
#Importing essential libraries
import pandas as pd
import numpy as np
import csv
import json
from itertools import islice
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import pickle
import math
from sklearn.model_selection import train_test_split
import operator
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index


False

In [65]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
docs_preprocessed = []

In [68]:
file1 = open("/content/drive/MyDrive/IR_Tweets_Data/twitter_base_preprocessed.pkl", "rb")
df = pickle.load(file1)
file1.close()

In [69]:
df.head()

,author id,created_at,geo,country,country_code,place_full_name,place_name,place_type,id,lang,...,source,tweet,hashtags,sensitive,urls,context_text,context_probability,context_type,Preprocessed_Data,Date_Only
0,1952510090.0,2022-02-14 00:11:44+00:00,,,,,,,1493015060943454208.0,en,...,Twitter for Android,ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸...,"CommonSenseGunLaws,GunControlNow,GunSafes,GunS...",False,https://t.co/LjkEup24Dk,,0.0,,[ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸...,2022-02-14
1,20453105.0,2022-02-14 00:04:56+00:00,,,,,,,1493013352938934272.0,en,...,Twitter for Android,"It's too bad these guys are afraid of needles,...","ClownConvoy,FreeDumbConvoy,OttawaOccupied",False,https://t.co/MmFzuFjIDR,,0.0,,"[bad, guy, afraid, needle, twinrix, would, pre...",2022-02-14
2,1492660754998247424.0,2022-02-14 00:39:55+00:00,,,,,,,1493022155239534336.0,en,...,Twitter for Android,Cowboy dressed as #Furries now available at th...,"Furries,RamRanch,OttawaOccupied",False,https://t.co/GBuBCUtpXe,,0.0,,"[cowboy, dressed, furries, available, ramranch...",2022-02-14
3,117812637.0,2022-02-14 00:20:05+00:00,,,,,,,1493017163485044736.0,en,...,Twitter for iPhone,We blocked these trucks from entering the dntn...,"Riverside,OttawaOccupied",False,https://t.co/KzLkBZvjgD,,0.0,,"[blocked, truck, entering, dntn, core, riversi...",2022-02-14
4,1490044524604928000.0,2022-02-14 00:10:26+00:00,,,,,,,1493014734903382016.0,en,...,Twitter for iPhone,Krista is very pleased with how the @RCMPONT r...,"FluTruxKlanGoHome,OttawaOccupied,kkkonvoy,Otta...",False,https://t.co/kAm5KNugdA,,0.0,,"[krista, pleased, rcmpont, responded, maskless...",2022-02-14


In [70]:
#Storing file name and data
total_documents = 0
path = '/content/drive/MyDrive/Tweelink_Articles_Processed'
for filename in glob(os.path.join(path, '*')):
   with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
     filename = os.path.basename(f.name)
     data = json.load(f)
     print(data)
     docs_preprocessed.append({'Name':filename, 'Data':data})
     total_documents+=1
print(total_documents)

{'S. No.': 'S. No. ', 'Base Hashtag': 'Base Hashtag', 'Link': 'Link', 'Date': 'Date', 'Location': 'Location', 'Title': 'Title', 'Body': 'Body', 'Body_processed': ['title', 'body']}
{'S. No.': '1', 'Base Hashtag': 'narendramodi', 'Link': 'https://zeenews.india.com/india/kanpur-dehat-surrounding-areas-are-going-to-bless-bjp-yet-again-pm-narendra-modi-2436450.html', 'Date': '2022-02-14', 'Location': 'Kanpur, India', 'Title': 'Kanpur Dehat, surrounding areas are going to bless BJP yet again: PM Narendra Modi', 'Body': 'Prime Minister Narendra Modi on Monday (February 14, 2022) addressed a public rally in the Kanpur Dehat region of Uttar Pradesh. In the rally, PM Modi exuded confidence and said that BJP government is inevitable, Yogi is coming back strongly in the state. PM Modi went on to say that the second phase of UP polls, which is currently underway, has cleared four things "BJP government is inevitable, Yogi is coming back strongly. Secondly, all castes, voters of all sections withou

In [71]:
docs_preprocessed[1]

{'Data': {'Base Hashtag': 'narendramodi',
  'Body': 'Prime Minister Narendra Modi on Monday (February 14, 2022) addressed a public rally in the Kanpur Dehat region of Uttar Pradesh. In the rally, PM Modi exuded confidence and said that BJP government is inevitable, Yogi is coming back strongly in the state. PM Modi went on to say that the second phase of UP polls, which is currently underway, has cleared four things "BJP government is inevitable, Yogi is coming back strongly. Secondly, all castes, voters of all sections without getting divided in villages and cities are voting for development and the BJP. Third, our mother, sisters, daughters, have taken up the BJP flag to ensure its victory, they want safety and dignity. "Fourth, our Muslim sisters silently have made up their mind and coming out to bless Modi. They know those who are there for them in time in need is a real friend," said PM Modi in Kanpur. Prime Minister is in Uttar Pradesh today and is going to campaign for 52 seats 

In [87]:
u_base_hashtag = input("Enter base hashtag: ")
u_time = input("Enter time: ")
u_location = input("Enter Location: ")

Enter base hashtag: TaylorSwift
Enter time: 2022-02-14
Enter Location: USA


In [88]:
import datetime
tweet_query = []
format = '%Y-%m-%d'
u_present_date = datetime.datetime.strptime(u_time, format)
u_prev_date = u_present_date - datetime.timedelta(days=1)
u_next_date = u_present_date + datetime.timedelta(days=1)
df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

In [89]:
for tweet in df_query['Preprocessed_Data']:
  tweet_query.extend(tweet)

In [90]:
tweet_query

['taylor',
 'swift',
 'man',
 'swifties',
 'taylorswift',
 'http',
 'tcohta3utaiec',
 'swifties',
 'taylorswift',
 'made',
 'kanyewestisoverparty',
 'taylorswift',
 'http',
 'tcowh5hbguoeh',
 'kanyewestisoverparty',
 'taylorswift',
 'valentine',
 's',
 'day',
 'anthem',
 'year',
 'lover',
 'taylor',
 'swift',
 'ð\x9f¤\x8d',
 'valentinesday',
 'valentinesday2022',
 'february14',
 'taylorswift',
 'lover',
 'http',
 'tco3ip9tx5vtw',
 'valentinesday',
 'valentinesday2022',
 'february14',
 'taylorswift',
 'lover',
 'favourite',
 'song',
 'ð\x9f\x98\x8dð\x9f\x98\xad',
 'thelakes',
 'taylorswift',
 'http',
 'tcopyc3qwnqiv',
 'thelakes',
 'taylorswift',
 'day',
 '101',
 'taylorswift13',
 'taylornation13',
 'fiancã©',
 'fan',
 'since',
 '1st',
 'uk',
 'show',
 'seen',
 'every',
 'one',
 'since',
 'play',
 'album',
 'road',
 'trip',
 'would',
 'mean',
 'world',
 'record',
 'short',
 'clip',
 'play',
 'weddingâ\x99¥ï¸\x8f',
 'taylorswiftâ',
 'â',
 'â',
 'â',
 'â',
 'â',
 'swifties',
 'taylorswift

In [91]:
import operator

def find_relevant_documents(docs_preprocessed, processed_query):
  jaccard_coefficients = {}
  for document in docs_preprocessed:
    doc_text = set(document['Data']['Body_processed'])
    query = set(processed_query)
    jaccard_coefficients[document['Name']] = len(doc_text.intersection(query))/len(doc_text.union(query))
  relevant_docs = list( sorted(jaccard_coefficients.items(), key=operator.itemgetter(1),reverse=True))[:20]
  return relevant_docs

def find_relevant_documents_2(docs_preprocessed, processed_query):
  jaccard_coefficients = {}
  for document in docs_preprocessed:
    doc_text = set(document['Data']['Body_processed'])
    query = set(processed_query)
    jaccard_coefficients[document['Name']] = len(doc_text.intersection(query))
  relevant_docs = list( sorted(jaccard_coefficients.items(), key=operator.itemgetter(1),reverse=True))[:20]
  return relevant_docs

In [95]:
find_relevant_documents(docs_preprocessed, tweet_query)

[('vaccine_299.json', 0.0438037149986138),
 ('UkraineRussiaCrisis_261.json', 0.03860691144708423),
 ('TaylorSwift_32.json', 0.03749267721148213),
 ('narendramodi_256.json', 0.03223981900452489),
 ('UkraineRussiaCrisis_264.json', 0.03170189098998888),
 ('hijab_286.json', 0.031509596104268116),
 ('IPL_274.json', 0.030214139043707833),
 ('TaylorSwift_38.json', 0.029982363315696647),
 ('TaylorSwift_31.json', 0.029938362195479896),
 ('TaylorSwift_187.json', 0.029618768328445746),
 ('TaylorSwift_181.json', 0.02918632075471698),
 ('UkraineRussiaCrisis_269.json', 0.029081485747192628),
 ('TaylorSwift_133.json', 0.02901915264074289),
 ('hijab_284.json', 0.029014084507042254),
 ('TaylorSwift_132.json', 0.028596961572832886),
 ('TaylorSwift_89.json', 0.026947861745752782),
 ('hijab_285.json', 0.026870846576134064),
 ('UkraineRussiaCrisis_270.json', 0.026808154146886344),
 ('IPL_276.json', 0.02675196277987787),
 ('UkraineRussiaCrisis_14.json', 0.02553916004540295)]

In [93]:
choice = "Y"
while choice == 'y' or choice == 'Y':
  query = input("\nEnter your query: ")
  processed_query = text_preprocess(query)
  print("Preprocessed query: ", processed_query)

  relevant_docs = find_relevant_documents(docs_preprocessed, processed_query)

  print("\nTop 5 relevant documents are: \n")
  for docs in relevant_docs:
    print(f"Doc name: {docs[0]} : jaccard coefficient: {docs[1]}")

  choice = input("\n\nDo you want to check for more queries (Y/N): ")

KeyboardInterrupt: ignored